## Ｓlot Game 機台自動生成輪帶表

<table class="colab-tcn-buttons" align="left">
  <td>
    <a href="https://colab.research.google.com/github/iinoshirozheng/Slot_Generator/blob/main/slot_generate.ipynb" target="_blank" ><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 上面執行</a>
  </td>
  <td>
    <a href="https://github.com/iinoshirozheng/Slot_Generator/blob/main/slot_generate.ipynb" target="_blank" ><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在我的 GitHub 上查看源碼</a>
  </td>
</table>

請注意以下幾點 (如果是要跑測試資料則可以不需要匯入檔案)：  
- 使用 Google CoLab 時，因為資料全部是在 Google 主機上跑，所以要把生成的資料上傳：  

   → 到左邊選單找到 檔案  

   → 找到 slot_genarator/data 檔案  

   → 將辨識出來的所有 csv 檔案丟進去  

   → 調整需要的參數到需要的值  

---
### 調整輸入需要的參數

In [137]:
#
# 加入需要的 Library
#
import numpy as np
import pandas as pd
import os
from datetime import datetime

#
# 需要調整的參數
#

#
# csv 資料夾存放位置
#
dir_path = 'data'

#
#  定義標籤
# 
labels = ["A","J", "Q", "K", "Boy", "Girl" , "Wild", "Scatter", "Carpet"]

#
# 生成輪帶表的長度
# 
generate_len = 200

#
# 生成輪帶表的數量
# 
generate_num = 5

#
# 限制是否全部都有才接， 如果打開 FREE GAME 大機率無法接起來
#
limit = True

#
# 設定均方根誤差值多少 ％
#
mse_error = 1

#
# 設定均方根誤差值多少 ％
#
mse_continue_error = 2

In [138]:
#
# 首先檢查是否在 Google CoLab 上執行
#

IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    print('Running on Google CoLab!')
else:
    print('Not running on Google CoLab!')

#
# 下載必須要用的 packages 以及 repository
#
if IN_COLAB:
    dir_path = '/content/slot_generator/data'
    if not os.path.exists('/content/slot_generator/'):
        print("Repo not cloned yet. Do it now!")
        !git clone https://github.com/iinoshirozheng/Slot_Generator.git /content/slot_generator/
        
    else:
        print("Repository already cloned!")

Not running on Google CoLab!


In [139]:
#
# 讀取資料中的所有生成 CSV 檔案並且合併成一個資料集
#
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if f.endswith('.csv')]
if len(file_list) > 1 :
    file_list = [file for file in file_list if 'test.csv' in file]
df_list = []
for file in file_list:
    df = pd.read_csv(file)
    df_list.append(df)
data = pd.concat(df_list)

### 將資料印出來確認一下有沒有正確

In [140]:
header = list(data.filter(regex='^L').columns)
data

,Round,Money,Mode,L11,L12,L13,L21,L22,L23,L31,...,A11,A12,A21,A22,A31,A32,A41,A42,A51,A52
0,1,47869,NORMAL,Carpet,Q,Carpet,Carpet,Scatter,J,Boy,...,NaN,NaN,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,47835,NORMAL,K,Boy,Q,K,Scatter,J,K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q,A
2,3,47785,NORMAL,Girl,A,K,Carpet,Scatter,J,Scatter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,47843,NORMAL,K,Boy,A,J,Q,Wild,Scatter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,47825,NORMAL,K,Boy,Q,K,A,K,Girl,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,678,47261,NORMAL,Carpet,Q,Carpet,Boy,K,Wild,J,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2126,679,47211,NORMAL,J,Girl,A,J,Carpet,Scatter,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN
2127,680,47193,NORMAL,K,K,J,Boy,K,A,J,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2128,681,47153,NORMAL,A,J,Girl,Q,Scatter,J,Scatter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 新增一個字典將 Mode 分開，並且取出只需要的部分 (Label)

In [141]:
#
# 新增一個字典將 MODE 分開
#
mode_data = {}
for mode in df['Mode']:
    mode_df = df[df['Mode'] == mode].copy()
    mode_df = mode_df.drop('Mode', axis = 1)
    # Data 轉成只有Label
    if type(mode) is str : mode_data[mode] = mode_df.filter(regex='^L')

### 將資料轉成數字，以便之後轉換成矩陣方便計算

In [142]:
#
# 所有資料轉成編號(表格的 index)，用於之後方便計算
#

for mode in mode_data.keys():
    mode_data[mode] = mode_data[mode].applymap(lambda x: labels.index(x) if x in labels else -1)

### 將資料轉成三維矩陣 (資料筆數, 輪帶數量, 輪帶 Label 數量)

In [143]:
#
# 獲得資料形狀
#
data_size = header[-1].split('L')[-1]
size_row = int(data_size[0])
size_column = int(data_size[1])
#
# 轉成矩陣
#
mode_array_data = {}
for mode in mode_data:
    mode_array_data[mode] = mode_data[mode].to_numpy().reshape(-1, size_row, size_column)




### 查看轉換成矩陣的輪代表是否正確

In [144]:
#
# 查看 輪帶表轉成的資料
#

for mode in mode_array_data.keys():
    print('='*50)
    print(f'Mode : {mode},\tShape : {mode_array_data[mode].shape}')
    print('='*50)
    for row in mode_array_data[mode]:
        for column in row:
            print(column, end='    ')
        print()

Mode : NORMAL,	Shape : (1912, 5, 3)
[8 2 8]    [8 7 1]    [4 8 2]    [8 0 2]    [0 1 2]    
[3 4 2]    [3 7 1]    [3 5 0]    [1 1 4]    [0 1 1]    
[5 0 3]    [8 7 1]    [7 0 4]    [8 0 2]    [0 1 2]    
[3 4 0]    [1 2 6]    [7 0 4]    [3 2 0]    [0 0 4]    
[3 4 2]    [3 0 3]    [5 8 3]    [1 8 1]    [0 1 2]    
[5 2 2]    [8 4 2]    [0 8 3]    [5 2 1]    [3 8 8]    
[2 5 0]    [7 0 5]    [1 0 8]    [5 4 0]    [2 3 8]    
[3 4 7]    [4 3 0]    [8 3 4]    [3 4 2]    [0 2 5]    
[0 5 2]    [3 0 5]    [2 5 0]    [1 4 3]    [4 3 4]    
[3 5 2]    [4 1 0]    [2 2 7]    [4 8 0]    [1 1 1]    
[0 7 1]    [3 2 2]    [4 8 2]    [1 4 8]    [2 3 8]    
[8 0 2]    [0 5 2]    [1 3 6]    [3 4 2]    [8 0 1]    
[2 1 5]    [3 6 0]    [8 1 8]    [2 8 1]    [2 1 8]    
[2 1 5]    [3 3 2]    [3 6 1]    [8 1 4]    [3 4 0]    
[2 2 5]    [6 2 5]    [4 7 0]    [3 8 1]    [3 4 0]    
[2 4 1]    [2 8 6]    [1 6 2]    [4 0 1]    [2 3 1]    
[1 5 1]    [3 4 3]    [0 3 3]    [6 0 5]    [0 2 5]    
[0 7 8]    [

### 開始試串輪帶表

In [145]:
#
# 將每個 labels 分類並計算數量
#
sigma_2 = 0.045500264
sigma_3 = 0.0026997961 
def unique_data(label_chain):
    label_arr, counts = np.unique(label_chain, axis=0, return_counts=True)
    # Find the indices of label categories with more than filter percent item
    # 正態分佈 99.7 % 以外都濾掉
    indices = np.where(counts/float(sum(counts)) > sigma_3)[0]

    # Sort the label categories by count (from high to low) and keep only those with more than one item
    filtered_indices = np.argsort(-counts[indices])
    # filtered_indices = np.argsort(indices)
    filtered_counts = counts[indices][filtered_indices]
    filtered_labels = [label_arr[i] for i in indices[filtered_indices]]

    for label, count in zip(filtered_labels, filtered_counts):
        print(f"{label} appears {count} items.\t{count/sum(counts)*100:.2f}% ({count}/{sum(filtered_counts)})")
    
    return filtered_labels, filtered_counts

def transform_matrix(label_chain):
    transform_arr = np.zeros((len(labels), len(labels)), dtype=float)
    for data,count in zip(label_chain[0], label_chain[1]):
        for i in range(len(data)-1):
            transform_arr[data[i]][data[i+1]] += float(count)
    # Normalize the rows of the array
    row_sums = transform_arr.sum(axis=1)
    nonzero_row_indices = row_sums.nonzero()[0]
    transform_arr[nonzero_row_indices] /= row_sums[nonzero_row_indices][:, np.newaxis]
    return transform_arr

def generate_label(label_chain, list_len = 0, limit = True):
    next_len = list_len
    if not list_len: limit = True
    if limit : list_len = 0
    sorted_labels = label_chain[0]
    matrix = transform_matrix(label_chain)
    # 選一個連續 label 並轉成列表
    random = np.random.choice(len(sorted_labels))
    count = 0

    gen_list = []
    for label in sorted_labels[random]:
        gen_list.append(label)
    check_list = [True for i in range(len(sorted_labels))] if limit else [False]
    if limit : check_list[random] = False
    same_rate = 1.0
    pre_list_len = 0
    is_last_label = False if limit else True
    # 當小於指定長度就一直生成
    while (sum(check_list) or (len(gen_list) <  list_len) or not is_last_label):

        # 抓取最後一個並且映射到轉置矩陣之中
        current_arr = matrix[gen_list[-1]]

        # 或取下一個要接的 LIST 的索引值
        next_index = np.random.choice(len(current_arr), p=current_arr)

        if(next_index == gen_list[-1]):
            same_rate *= matrix[gen_list[-1]][next_index]
            if same_rate < sigma_2: continue
        else:
            same_rate = 1

        if limit :
            if len(match := np.where(np.all(sorted_labels == np.array([gen_list[-2], gen_list[-1], int(next_index)]), axis=1))[0]) \
                and any(np.array_equal([gen_list[-1], int(next_index)], label[:2]) for label in sorted_labels):
        
                gen_list.append(int(next_index))
                check_list[match[0]] = False

            if len(gen_list) > 1000:
                gen_list.clear()
                random = np.random.choice(len(sorted_labels))
                for label in sorted_labels[random]:
                    gen_list.append(label)
                check_list = [True for i in range(len(sorted_labels))]
                check_list[random] = False
                count+=1
            if len(gen_list) == pre_list_len: count += 1
            if count > len(sorted_labels)**2:
                return generate_label(label_chain, generate_len if next_len == 0 else next_len, limit=False)
            

            if len(match := np.where(np.all(sorted_labels == np.array([gen_list[-1], gen_list[0], gen_list[1]]), axis=1))[0]) \
            and len(match := np.where(np.all(sorted_labels == np.array([gen_list[-2], gen_list[-1], gen_list[0]]), axis=1))[0]):
                is_last_label = True
            else:
                is_last_label = False
        else:
            gen_list.append(int(next_index))
            
        
        pre_list_len = len(gen_list)

    return gen_list, cucular_chain_mse(label_chain, gen_list), cucular_mse(label_chain, gen_list), limit

def cucular_mse(label_chain, lists):
    appear_rate = [0 for i in range(len(labels))]

    for label, count in zip(label_chain[0], label_chain[1]):
        for _ in range(count):
            for element in label:
                appear_rate[element] +=1
    appear_rate = np.array(appear_rate, dtype =float)
    appear_rate /= sum(appear_rate)


    rate = [0 for i in range(len(labels))]
    for element in lists:
        rate[element] += 1
    rate = np.array(rate, dtype =float)
    rate /= sum(rate)
    return np.square(rate*100 - appear_rate*100).mean(axis=0)

def cucular_chain_mse(label_chain, lists):

    rate = [0 for i in range(len(label_chain[0]))]

    appear_rate = label_chain[1] / sum(label_chain[1])
    if len(match := np.where(np.all(label_chain[0] == np.array([lists[-1], lists[0], lists[1]]), axis=1))[0]):
        for element in match:
            rate[element] += 1
    if len(match := np.where(np.all(label_chain[0] == np.array([lists[-2], lists[-1], lists[0]]), axis=1))[0]):
        for element in match:
            rate[element] += 1

    for i in range(len(lists) - 2):
        if len(match := np.where(np.all(label_chain[0] == np.array(lists[i:i+3]), axis=1))[0]):
            for element in match:
                rate[element] += 1

    rate = np.array(rate, dtype =float)
    rate /= sum(rate)
    return np.square(rate*100 - appear_rate*100).mean(axis=0)

def generate_data(unique_dataset, generate_num, generate_len, mse_chain_error, mse_error, limit):
    start_limit = limit
    generate_chain = {}
    for mode in unique_dataset.keys():
        chain = []
        for chain_num in range(len(unique_dataset[mode])):
            gen_labels = []
            print(f'Mode : {mode}, Chain list number : {chain_num}')
            count_limit = 0
            temp_limit = limit
            while len(gen_labels) < generate_num:
                label_chain = unique_dataset[mode][chain_num]
                gen, gen_chain_mse, gen_mse, gen_limit = generate_label(label_chain, generate_len, temp_limit)
                if gen_limit != limit: 
                    count_limit+=1
                # print(gen_chain_mse, gen_mse)
                    if count_limit > 100:
                        temp_limit = gen_limit
                else: count_limit = 0
                    # if mse_error < 4.5:  mse_error= 4.5
                # if limit and generate_len == 0: len_flag = True
                # elif len(gen) <= generate_len : len_flag = True
                if gen_mse <= mse_error and len(gen) <=generate_len  and gen_chain_mse <= mse_chain_error: 
                    print(f'generate lable list {len(gen)}, Chain MSE : {gen_chain_mse:.2f} % , MSE : {gen_mse:.2f} %')
                    print(gen)
                    gen_labels.append((gen,gen_chain_mse, gen_mse))
            chain.append(gen_labels)
            print()
        generate_chain[mode] = chain
    return generate_chain


def print_result(generate_chain):
    print('Label 對照編號 : [', end='')
    for i, label in enumerate(labels):
        print(f' {label}:{i} ', end='')
    print(']')
    print()
    for mode in generate_chain.keys():
        for chain_num in range(len(generate_chain[mode])):
            print(f'模式 (Mode) : {mode}, 輪帶編號 : L{chain_num}')
            for index, result in enumerate(generate_chain[mode][chain_num]):
                print(f'\n{index+1}. Symbol 誤差值 : {result[1]:.2f}, 誤差值 : {result[2]:.2f}, 輪帶長度 : {len(result[0])}\n生成輪帶 : {result[0]}')
            print()
            print()

def generate_symbols_excel(unique_dataset):
    for mode in unique_dataset.keys():
        now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        with pd.ExcelWriter(f'各Symbol比例_{mode}_{now}.xlsx') as writer:
            for chain_num, chain in enumerate(unique_dataset[mode]):
                label_list = []
                count_list = []
                persent_list = []
                for label, count in zip(chain[0],chain[1]):
                    label_name = '[ '
                    for i,index in enumerate(label):
                        if i : label_name+=', '
                        label_name += f'{labels[index]}'
                    label_name += ' ]'

                    label_list.append(label_name)
                    count_list.append(count)
                    persent_list.append(f'{count/sum(chain[1]):.5f} %  ({count}/{sum(chain[1])})')
                    
                chain_dataframe = {'Symbol': label_list, '數量': count_list, '各Symbol的比例': persent_list}
                pd.DataFrame(chain_dataframe).to_excel(writer, sheet_name=f'第 {chain_num+1} 條輪帶表', index=False)

def save_generated_labels_toExcel(generate_chain):
    for mode in generate_chain.keys():
        now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        with pd.ExcelWriter(f'生成的輪帶表_{mode}_{now}.xlsx') as writer:
            for chain_num in range(len(generate_chain[mode])):
                label_len_list = []
                chain_mse_list = []
                mse_list = []
                generate_labels = []
                # print(f'模式 (Mode) : {mode}, 輪帶編號 : L{chain_num}')
                for result in generate_chain[mode][chain_num]:
                    # print(f'\n{index+1}. 誤差值 : {result[1]:.2f}, 輪帶長度 : {len(result[0])}\n生成輪帶 : {result[0]}')
                    label = result[0]
                    chain_mse = result[1]
                    mse = result[2]


                    label_len_list.append(len(label))

                    labels_name = '[ '
                    for comma,symbol in enumerate(label):
                        if comma : labels_name +=', '
                        labels_name += f'{labels[symbol]}'
                    labels_name += ' ]'

                    generate_labels.append(labels_name)
                    chain_mse_list.append(chain_mse)
                    mse_list.append(mse)
                    
                chain_dataframe = {'長度': label_len_list, '排列誤差 (arrangement MSE)': chain_mse_list, '各Symbol的出現率誤差 (MSE)': mse_list, '生成的輪帶表':generate_labels}
                chain_dataframe = pd.DataFrame(chain_dataframe).sort_values(by='長度', ascending=True)
                chain_dataframe.to_excel(writer, sheet_name=f'第 {chain_num+1} 條輪帶表', index=False)


### 計算每個 Label chain 個別出現總數

In [146]:
#
# 計算每個 Label 出現的個數，並存在 unique_dataset 中
#

unique_dataset = {}

for mode in mode_array_data.keys():
    print(mode, " : \n")
    chain = []
    for chain_num in range(mode_array_data[mode].shape[1]):
        chain.append(unique_data(mode_array_data[mode][:, chain_num]))
        print(f"L{chain_num}: {len(chain[-1][0])} categories with more than one item.")
        print()
    unique_dataset[mode] = chain


NORMAL  : 

[5 2 2] appears 64 items.	3.35% (64/1907)
[1 8 0] appears 63 items.	3.29% (63/1907)
[4 1 8] appears 52 items.	2.72% (52/1907)
[5 2 3] appears 47 items.	2.46% (47/1907)
[7 8 2] appears 43 items.	2.25% (43/1907)
[4 0 1] appears 42 items.	2.20% (42/1907)
[1 1 2] appears 42 items.	2.20% (42/1907)
[3 1 0] appears 41 items.	2.14% (41/1907)
[4 3 3] appears 40 items.	2.09% (40/1907)
[7 8 0] appears 38 items.	1.99% (38/1907)
[2 8 0] appears 37 items.	1.94% (37/1907)
[3 4 7] appears 36 items.	1.88% (36/1907)
[2 3 1] appears 36 items.	1.88% (36/1907)
[2 2 4] appears 36 items.	1.88% (36/1907)
[8 2 1] appears 36 items.	1.88% (36/1907)
[8 0 7] appears 36 items.	1.88% (36/1907)
[7 1 8] appears 35 items.	1.83% (35/1907)
[0 8 4] appears 35 items.	1.83% (35/1907)
[8 3 4] appears 35 items.	1.83% (35/1907)
[2 5 0] appears 35 items.	1.83% (35/1907)
[8 0 2] appears 34 items.	1.78% (34/1907)
[2 1 5] appears 34 items.	1.78% (34/1907)
[5 0 3] appears 34 items.	1.78% (34/1907)
[3 3 1] appears 34 ite

### 生成各 Symbol 比例的 EXCEL

In [147]:
generate_symbols_excel(unique_dataset)

### 生成並擬和輪帶表

In [148]:
# 生成設定的輪帶表
# generate_num = 4 , generate_len <= 200 , mse_error < 1.0, limit = False
generate_chain = generate_data(unique_dataset, generate_num=generate_num, generate_len=generate_len, mse_chain_error=mse_continue_error ,mse_error =mse_error, limit = limit)


Mode : NORMAL, Chain list number : 0
generate lable list 164, Chain MSE : 0.59 % , MSE : 0.82 %
[2, 1, 1, 2, 3, 4, 7, 8, 0, 8, 4, 1, 8, 0, 8, 4, 1, 8, 2, 8, 0, 3, 5, 2, 2, 5, 2, 3, 4, 2, 1, 5, 1, 8, 3, 4, 7, 8, 2, 8, 0, 3, 5, 2, 3, 4, 0, 1, 5, 1, 8, 0, 2, 5, 2, 3, 1, 0, 5, 2, 3, 4, 3, 3, 1, 0, 5, 2, 3, 1, 7, 3, 4, 3, 3, 1, 0, 5, 2, 2, 5, 2, 3, 1, 0, 5, 2, 2, 4, 1, 8, 0, 3, 4, 2, 1, 5, 0, 7, 8, 0, 7, 8, 2, 8, 0, 8, 4, 1, 8, 2, 1, 5, 0, 3, 4, 0, 1, 5, 1, 8, 0, 7, 8, 2, 1, 1, 2, 3, 1, 7, 3, 4, 7, 8, 2, 1, 5, 1, 8, 2, 1, 5, 0, 3, 4, 3, 3, 1, 7, 3, 4, 3, 3, 1, 0, 5, 2, 2, 5, 0, 7, 1, 8]
generate lable list 182, Chain MSE : 0.85 % , MSE : 0.91 %
[2, 3, 1, 0, 5, 2, 3, 1, 0, 5, 2, 2, 4, 1, 8, 2, 1, 5, 1, 8, 2, 8, 0, 8, 4, 1, 8, 3, 4, 0, 1, 5, 0, 3, 5, 2, 3, 1, 7, 3, 4, 7, 8, 2, 8, 0, 2, 5, 2, 3, 4, 2, 1, 1, 2, 3, 4, 0, 1, 5, 0, 3, 5, 2, 3, 4, 2, 1, 5, 1, 8, 0, 2, 5, 2, 2, 5, 2, 3, 4, 0, 1, 5, 1, 8, 2, 8, 0, 2, 5, 2, 2, 5, 2, 2, 4, 1, 8, 2, 1, 5, 0, 7, 8, 0, 3, 4, 0, 1, 5, 1, 8, 0, 3, 4, 7, 8, 

### 印出結果

In [149]:
print_result(generate_chain)

Label 對照編號 : [ A:0  J:1  Q:2  K:3  Boy:4  Girl:5  Wild:6  Scatter:7  Carpet:8 ]

模式 (Mode) : NORMAL, 輪帶編號 : L0

1. Symbol 誤差值 : 0.59, 誤差值 : 0.82, 輪帶長度 : 164
生成輪帶 : [2, 1, 1, 2, 3, 4, 7, 8, 0, 8, 4, 1, 8, 0, 8, 4, 1, 8, 2, 8, 0, 3, 5, 2, 2, 5, 2, 3, 4, 2, 1, 5, 1, 8, 3, 4, 7, 8, 2, 8, 0, 3, 5, 2, 3, 4, 0, 1, 5, 1, 8, 0, 2, 5, 2, 3, 1, 0, 5, 2, 3, 4, 3, 3, 1, 0, 5, 2, 3, 1, 7, 3, 4, 3, 3, 1, 0, 5, 2, 2, 5, 2, 3, 1, 0, 5, 2, 2, 4, 1, 8, 0, 3, 4, 2, 1, 5, 0, 7, 8, 0, 7, 8, 2, 8, 0, 8, 4, 1, 8, 2, 1, 5, 0, 3, 4, 0, 1, 5, 1, 8, 0, 7, 8, 2, 1, 1, 2, 3, 1, 7, 3, 4, 7, 8, 2, 1, 5, 1, 8, 2, 1, 5, 0, 3, 4, 3, 3, 1, 7, 3, 4, 3, 3, 1, 0, 5, 2, 2, 5, 0, 7, 1, 8]

2. Symbol 誤差值 : 0.85, 誤差值 : 0.91, 輪帶長度 : 182
生成輪帶 : [2, 3, 1, 0, 5, 2, 3, 1, 0, 5, 2, 2, 4, 1, 8, 2, 1, 5, 1, 8, 2, 8, 0, 8, 4, 1, 8, 3, 4, 0, 1, 5, 0, 3, 5, 2, 3, 1, 7, 3, 4, 7, 8, 2, 8, 0, 2, 5, 2, 3, 4, 2, 1, 1, 2, 3, 4, 0, 1, 5, 0, 3, 5, 2, 3, 4, 2, 1, 5, 1, 8, 0, 2, 5, 2, 2, 5, 2, 3, 4, 0, 1, 5, 1, 8, 2, 8, 0, 2, 5, 2, 2, 5, 2, 2, 4, 1

### 生成輪帶表的 EXCEL

In [150]:
save_generated_labels_toExcel(generate_chain)

### 嘗試其他組合

In [ ]:
generate_chain_1 = generate_data(unique_dataset, generate_num=1, generate_len=200, mse_chain_error=0.3 ,mse_error =0.3, limit = False)